##  To do:
- create gis identity layer of different polygons
- apportion se data to new polygons
    - join se_by_parcel.csv to parcel shapes

In [16]:
import arcpy
from arcpy import env
import os
import numpy as np
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False) 

In [17]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "scratch.gdb"]
gdb = os.path.join(outputs[0], outputs[1])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

In [18]:
# msf = pd.DataFrame.spatial.from_featureclass(r"E:\Projects\REMM-Analysis-Tools\Growth\Outputs\scratch.gdb\Master_Summary_Features_v2")
# parcel_shp = pd.DataFrame.spatial.from_featureclass(r"E:\Projects\REMM-Manage-Base-Year-Data\Current_Inputs\remm_base_year.gdb\parcels")[['parcel_id', 'SHAPE']]
# se = pd.read_csv(r"E:\Projects\REMM-Analysis-Tools\Growth\Outputs\SE_by_PARCEL.csv")

In [19]:
# msf2 = msf[['OBJECTID', 'Acres', 'CO_NAME', 'Id', 'CityArea', 'AreaName', 'AreaType', 'SHAPE']].copy()
# msf2.columns = ['UID','ACRES', 'COUNTY_NAME', 'INFILL1990', 'CITY_AREA', 'CENTER_NAME', 'CENTER_TYPE', 'SHAPE']
# msf2.fillna({'CENTER_NAME':'Non-Center', 'CENTER_TYPE':'Non-Center', 'CITY_AREA':'Unincorporated'}, inplace=True)
# msf2.spatial.to_featureclass(location=os.path.join(gdb,'Master_Summary_Features'),sanitize_columns=False)

In [20]:
# parcel_se = parcel_shp.merge(se, on='parcel_id', how='left')
# parcel_se.spatial.to_featureclass(location=os.path.join(gdb,'parcels_se'),sanitize_columns=False) 

In [21]:
# taz = pd.DataFrame.spatial.from_featureclass(r'E:\Projects\REMM-Process-SE-Output-For-Web\Inputs\TAZ_900.gdb\TAZ_UTM12')[['SA_TAZID', 'ACRES', 'DEVACRES', 'SHAPE', 'CO_FIPS', 'CityArea']].copy()
# taz.rename({'SA_TAZID':'TAZID_900'}, inplace=True, axis=1)
# se_taz = pd.read_csv(r"E:\Projects\REMM-Analysis-Tools\Growth\Outputs\SE_Density_TAZ900.csv")
# x = taz.merge(se_taz, on='TAZID_900', how='left')
# x.spatial.to_featureclass(location=os.path.join(gdb,'SE_Density_by_TAZ900'),sanitize_columns=False)


In [22]:
MSF_SE['CENTER_TYPE'].value_counts()

Non-Center             337
City Center            177
Neighborhood Center    110
Industrial District     54
Employment District     42
Urban Center            39
Special District        21
Metropolitan Center      4
Educational Center       3
Name: CENTER_TYPE, dtype: int64

In [23]:
# -- created identity layer using 4 geographies
# -- apportion tool was used to apply parcel se data to new identity shape

MSF_SE = pd.DataFrame.spatial.from_featureclass(r'E:\Projects\REMM-Analysis-Tools\Growth\Outputs\scratch.gdb\Master_Summary_Features_with_SE_v2')

# rename
MSF_SE.rename({'OBJECTID':'UID', 'CO_NAME':'COUNTY_NAME', 'Id':'INFILL1990', 'CityArea':'CITY_AREA', 'AreaName':'CENTER_NAME', 'AreaType':'CENTER_TYPE','NewSA':'WORKSHOP_AREA'}, inplace=True, axis=1)

# fill na's in parcel eq
MSF_SE.fillna({'CENTER_NAME':'Non-Center', 'CENTER_TYPE':'Non-Center', 'CITY_AREA':'Non-City-Area', 'WORKSHOP_AREA':'NA'}, inplace=True)
MSF_SE.replace("",{'CENTER_NAME':'Non-Center', 'CENTER_TYPE':'Non-Center', 'CITY_AREA':'Non-City-Area', 'WORKSHOP_AREA':'NA'}, inplace=True)
MSF_SE.replace(" ",{'CENTER_NAME':'Non-Center', 'CENTER_TYPE':'Non-Center', 'CITY_AREA':'Non-City-Area', 'WORKSHOP_AREA':'NA'}, inplace=True)

# round data to whole number
cols = ['sf_units_19','mf_units_19','residential_units_19','job_spaces_19','ind_jobs_19','rtl_jobs_19','off_jobs_19','ttl_jobs_19',
        'sf_units_50','mf_units_50','residential_units_50','job_spaces_50','ind_jobs_50','rtl_jobs_50','off_jobs_50','ttl_jobs_50']
MSF_SE[cols] = MSF_SE[cols].round().fillna(0)

# round acres
cols2 = ['ACRES','parcel_acres','vacant_acres_19', 'vacant_acres_50', 'vacant_devacres_19', 'vacant_devacres_50']
MSF_SE[cols2] = MSF_SE[cols2].round(1).fillna(0)


# Center type recode
center_type_lu = {'City Center': '3 City Center',
                'Non-Center': '0 Non-Center',
                'Neighborhood Center': '4 Neighborhood Center',
                'Industrial District': '8 Industrial District',
                'Employment District': '5 Employment District',
                'Urban Center': '2 Urban Center',
                'Special District': '7 Special District',
                'Metropolitan Center': '1 Metropolitan Center',
                'Educational Center': '6 Educational Center'}

MSF_SE['CENTER_TYPE'] = MSF_SE['CENTER_TYPE'].map(center_type_lu)

# subset columns
cols3 = ['UID','ACRES',
         'COUNTY_NAME', 'WORKSHOP_AREA', 'INFILL1990',
         'CITY_AREA', 'CENTER_NAME',
         'CENTER_TYPE', 'parcel_acres',
         'vacant_acres_19', 'vacant_acres_50', 
         'vacant_devacres_19', 'vacant_devacres_50',
         'sf_units_19','sf_units_50', 
         'mf_units_19','mf_units_50',
         'residential_units_19','residential_units_50',
         'job_spaces_19','job_spaces_50',
         'ind_jobs_19','ind_jobs_50',
         'rtl_jobs_19','rtl_jobs_50',
         'off_jobs_19','off_jobs_50',
         'ttl_jobs_19','ttl_jobs_50']

MSF_SE = MSF_SE[cols3]
MSF_SE.to_csv(os.path.join(outputs[0], 'REMM_Master_Summary_Table_2019_2050.csv'), index=False)
MSF_SE.head()

,UID,ACRES,COUNTY_NAME,WORKSHOP_AREA,INFILL1990,CITY_AREA,CENTER_NAME,CENTER_TYPE,parcel_acres,vacant_acres_19,vacant_acres_50,vacant_devacres_19,vacant_devacres_50,sf_units_19,sf_units_50,mf_units_19,mf_units_50,residential_units_19,residential_units_50,job_spaces_19,job_spaces_50,ind_jobs_19,ind_jobs_50,rtl_jobs_19,rtl_jobs_50,off_jobs_19,off_jobs_50,ttl_jobs_19,ttl_jobs_50
0,1,57.7,DAVIS,NA,0,Bountiful,Non-Center,0 Non-Center,4.1,0.7,0.0,0.7,0.0,2.0,6.0,0.0,0.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,107.8,DAVIS,NA,0,Centerville,Non-Center,0 Non-Center,3.6,3.5,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,310850.8,DAVIS,NA,0,Davis County,Non-Center,0 Non-Center,950.9,912.2,901.4,22.9,12.0,2.0,50.0,0.0,0.0,2.0,50.0,6.0,6.0,1.0,1.0,1.0,1.0,4.0,4.0,6.0,6.0
3,4,11.2,DAVIS,NA,0,Farmington,Non-Center,0 Non-Center,8.6,7.6,6.6,1.0,0.0,0.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,DAVIS,NA,0,Fruit Heights,Non-Center,0 Non-Center,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
